# Beampattern Notebook

Concepts to communicate:

1) Beampattern
2) Main Lobe
3) Sidelobes
4) Beam width
5) Source Level
6) Near-field vs far-field

A **transducer** is a device that takes an electrical signal and transforms it into mechanical motion. The audio speakers in computers, phones, or headphones are probably the transducers we interact with and use on a daily basis. These transducers use an electrical signal to cause the speaker surface to oscillate which then generates sound. The acoustic source in a sonar system does the same thing. A transducer can also take mechanical motion and transform it into an electrical signal (microphones in air and hydrophones in the water), but this section will focus only on the properties of acoustic transducers that are used to generate sound.

## Circular transducer

The most common man-made sound sources used in underwater acoustics are planer transducers which have flat surfaces which oscillate to produce sound. This is in part due to the fact that a flat transducer is relatively easy to manufacturer, but more importantly, the transducer transmits a majority of the sound in the direction perpendicular to the vibrating plane.  This is a useful property in many sonar systems and this directivity of the transmitted sound can help determine the location of an object (fish, bubble, submarine, etc.)

Note that we said the transducer directs a "majority" of the sound in one direction and not "all" of the sound. In fact, the transducer sends different levels of sound in many directions and the directional dependence of the sound is refered to as the transducer's **beam pattern**. To help understand the concept of a beam pattern and it's properties, we will focus initially on one particular, common type of tranducer: the circular piston. This type of transducer is often a cylindrical piece of piezoelectric material to which an electrical signal is applied such that it expands and contracts along the axis of the cylinder. The causes sound to radiate from the circular face of the transducer. The structure of the sound field from this type of transducer can be fairly complicated and there is not a simple, mathematical expression for the direction and strength of the transmitted sound. There are, however, approximate mathematical descriptions that can be accurately describe the transmitted sound under certain conditions. One of the most useful of these is to assume that the transducer an be approximated as a baffled circular piston. By "baffled," we mean that the face of the transducer is surrounded by an infinite, rigid surface. This simplifies the mathematics and allows us to calculate strength and direction of the transmitted field.

In the following, the face of the transducer will be in the x-y plane and the z-axis is perpendicular to the face. The radius of the tranducer is ***a*** and the field will be described in two ways; either a function of the distance, ***r***, from the center of the face and the angle, ***$\theta$***, between the direction of ***r*** and the -axis or as a function of distance from the plane of the transducer (range) and the perpendicular direction (cross-range.) 


In [15]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import j1  # Bessel function of the first kind
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd

In [16]:
# Constants
rho0 = 1
c = 1500
r = 10

# Initialize variables that will be updated in the interactive plot and used to find which
# angles correspond to 3, 6, and 10dB
theta = None
projInt = None
axisInt = None

# Create sliders for 'f' and 'a'
f_slider = widgets.IntSlider(min=1, max=50, step=1, value=1, description='Frequency (kHz)')
a_slider = widgets.FloatSlider(min=0.5, max=1, step=0.05, value=1, description='Aperture Radius (m)')

# Function to update plot based on widget input
def update_far_field_intensity_cartesian_polar_plot(f, a, plotType, plotScale):
    global theta, projInt, axisInt  # Declare the variables as global to allow updates

    omega = 2 * np.pi * f * 1000
    k = omega / c
    theta = np.arange(-90, 90.1, 0.1)
    u = k * a * np.sin(theta * np.pi / 180)

    # Projection Intensity calculation
    projInt = np.abs((rho0 * c * k * a * a / (2 * r)) * 
                     (2 * j1(u) / u)) ** 2

    # Handle division by zero at theta = 0
    ind = np.where(theta == 0)[0]
    axisInt = np.abs(rho0 * c * k * a * a / (2 * r)) ** 2
    projInt[ind] = axisInt

    plt.close('all')  # Close any existing plots

    if plotType == 'Cartesian':
        if plotScale == "Decibels":
            plt.plot(theta, 10 * np.log10(projInt / axisInt), linewidth=2)
            plt.ylim([-40, 0])
            plt.xlim([-90, 90])
            plt.xlabel('Angle (degrees)', fontsize=14)
            plt.ylabel('Beam Level (dB)', fontsize=14)
            plt.grid(True)
        else:
            plt.plot(theta, projInt / axisInt, linewidth=2)
            plt.ylim(10 ** (np.array([-40, 0]) / 10))
            plt.xlim([-90, 90])
            plt.xlabel('Angle (degrees)', fontsize=14)
            plt.ylabel('Beam Level', fontsize=14)
            plt.grid(True)
    
    elif plotType == 'Polar':
        if plotScale == "Decibels":
            plt.polar(theta * np.pi / 180, 10 * np.log10(projInt / axisInt), linewidth=2)
            ax = plt.gca()
            ax.set_ylim([-40, 0])
            ax.set_theta_zero_location('N')  # 'top'
            ax.set_thetalim([np.radians(-90), np.radians(90)])
            ax.set_title('Beam Pattern (dB)', fontsize=14)
        else:
            plt.polar(theta * np.pi / 180, projInt / axisInt, linewidth=2)
            ax = plt.gca()
            ax.set_ylim(10 ** (np.array([-40, 0]) / 10))
            ax.set_theta_zero_location('N')  # 'top'
            ax.set_thetalim([np.radians(-90), np.radians(90)])
            ax.set_title('Beam Pattern (Linear Scale)', fontsize=14)

# Create widgets for plot type and scale
plot_type_widget = widgets.Dropdown(
    options=['Cartesian', 'Polar'],
    value='Polar',
    description='Plot Type:'
)

plot_scale_widget = widgets.Dropdown(
    options=['Decibels', 'Linear'],
    value='Decibels',
    description='Scale:'
)

# Use interact to link widgets and plotting function
interact(
    update_far_field_intensity_cartesian_polar_plot,
    f=f_slider,
    a=a_slider,
    plotType=plot_type_widget,
    plotScale=plot_scale_widget,
)

interactive(children=(IntSlider(value=16, description='Frequency (kHz)', max=50, min=1), FloatSlider(value=1.0…

<function __main__.update_far_field_intensity_cartesian_polar_plot(f, a, plotType, plotScale)>

In [17]:
# Compute beam widths
ind3dB = np.where(10 * np.log10(projInt / axisInt) >= -3)[0]
ind6dB = np.where(10 * np.log10(projInt / axisInt) >= -6)[0]
ind10dB = np.where(10 * np.log10(projInt / axisInt) >= -10)[0]

bw3dB = np.abs(theta[ind3dB[0]])
bw6dB = np.abs(theta[ind6dB[0]])
bw10dB = np.abs(theta[ind10dB[0]])

# Define labels and values
bwLabels = ['down 3 dB', 'down 6 dB', 'down 10 dB']
bwValues = [bw3dB, bw6dB, bw10dB]

# Create a DataFrame
df = pd.DataFrame({
    'Beam Width (degrees)': bwValues,
    'Definition': bwLabels
})

# Print the DataFrame
print(df)

   Beam Width (degrees)  Definition
0                   1.3   down 3 dB
1                   1.8   down 6 dB
2                   2.3  down 10 dB


In [18]:
# Constants
rho0 = 1
c = 1500
r = 10

# Create sliders for 'f' and 'a'
f_slider = widgets.IntSlider(min=1, max=50, step=1, value=1, description='Frequency (kHz)')
a_slider = widgets.FloatSlider(min=0.5, max=1, step=0.05, value=1, description='Aperture Radius (m)')

# Function to update plot based on widget input
def update_far_field_intensity_x_y_plane_plot(f, a, plotScale):
    omega = 2 * np.pi * f * 1000
    k = omega / c

    plt.close('all')  # Close any existing plots

    # Calculations
    SL_dB = 180
    SL_lin = 10**(SL_dB / 20)
    U0 = SL_lin / (rho0 * c * k * a * a / 2)

    x = np.linspace(-r, r, 150)
    y = np.linspace(0, r, 100)
    X, Y = np.meshgrid(x, y)

    thetaXY = np.arctan2(X, Y)
    rXY = np.sqrt(X**2 + Y**2)
    u1 = k * a * np.sin(thetaXY)
    u1[u1 == 0] = np.nan  # Avoid division by zero

    projIntXY = np.abs((rho0 * c * k * U0 * a * a / (2 * rXY)) *
                    (2 * j1(u1) / u1))**2

    # Plotting
    plt.figure(2)
    if plotScale == 'Decibels':
        plt.imshow(10 * np.log10(projIntXY), extent=(x.min(), x.max(), y.min(), y.max()), origin='lower')
        plt.colorbar(label='Intensity (dB re 1μPa)')
        plt.clim(SL_dB - 90, SL_dB)
    else:
        plt.imshow(projIntXY / 1e6, extent=(x.min(), x.max(), y.min(), y.max()), origin='lower')
        plt.colorbar(label='Intensity (μPa²)')
        plt.clim(10**((SL_dB - 90) / 10), SL_dB)
        
    plt.xlabel('Cross Range (m)')
    plt.ylabel('Range (m)')
    plt.gca().set_aspect('auto')
    plt.show()

# Use interact to link widgets and plotting function
interact(
    update_far_field_intensity_x_y_plane_plot,
    f=f_slider,
    a=a_slider,
    plotScale=plot_scale_widget,
)

interactive(children=(IntSlider(value=16, description='Frequency (kHz)', max=50, min=1), FloatSlider(value=1.0…

<function __main__.update_far_field_intensity_x_y_plane_plot(f, a, plotScale)>